In [30]:

from time import sleep
import re
from collections import Counter
import cv2
import pytesseract
import preprocessing



In [41]:
# Path to Tesseract executable
TESSERACT_CMD = r'C:\Users\james\AppData\Local\Tesseract-OCR\tesseract.exe'
# Custom configuration for Tesseract OCR
CUSTOM_CONFIG = r'--oem 3 --psm 6'

pytesseract.pytesseract.tesseract_cmd = TESSERACT_CMD

In [117]:
def read_once(gscale, x, refined, grayscale_modes, scale_range=5):
    # Read the image from the saved path
    image_path = f"img/screenshot{x}.jpg"
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Failed to read image at {image_path}")
        return [0, 'Failed to read image']
    
    scale = grayscale_modes[gscale]
    print(f'Testing {scale} scale')
    gray = cv2.cvtColor(image, scale)
    noise = preprocessing.remove_noise(gray)
    canny = preprocessing.canny(noise)
    thresh = preprocessing.thresholding(canny)

    # Extract text from the processed image using Tesseract OCR
    text = pytesseract.image_to_string(thresh, config=CUSTOM_CONFIG)
    text = re.sub(r'\D', '', text)
    value = int(text) if text else 0
    print(value)
    refined.append(value)

    # Frequency calculation after all grayscale modes have been tested
    frequency = Counter(refined).most_common()
    most_common_value = frequency[0][0]

    # Retry with different grayscale modes if value is less than 1000 or greater than 10000
    if (most_common_value < 1000 or most_common_value > 10000) and gscale < scale_range - 1:
        return read_once(gscale + 1, x, refined, grayscale_modes, scale_range)
    
    return [most_common_value, 'Success' if gscale == 0 else 'Reviewed']

In [ ]:
grayscale_modes = [cv2.COLOR_BGR2Luv, cv2.COLOR_BGR2Lab, cv2.COLOR_BGR2XYZ, cv2.COLOR_BGR2GRAY, cv2.COLOR_RGB2LUV]
for x in range (1,26):
    scale_range = len(grayscale_modes)
    refined = []
    print(read_once(0, x, refined, grayscale_modes, scale_range))